In [ ]:
!pip install opencv-python
import cv2
import numpy as np
import torch
import torchvision
from torchvision import transforms

# Check device

# Load pretrained Mask R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# COCO class labels (80)
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
    'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
    'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana',
    'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table',
    'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock',
    'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


100%|██████████| 160M/160M [00:01<00:00, 89.7MB/s]


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device).eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
path="/content/video.mp4"
cap=cv2.VideoCapture(path)

print("Opened?", cap.isOpened())
print("Frame count:", cap.get(cv2.CAP_PROP_FRAME_COUNT))

if not cap.isOpened():
    print(f"Error: Could not open video file at {path}")
    exit() # Exit if video not opened

Opened? True
Frame count: 308.0


In [ ]:
import os
os.path.getsize('video.mp4') / 1024

12429.5263671875

In [ ]:
import cv2

cap = cv2.VideoCapture('/content/video.mp4')  # adjust path if needed
print("Opened:", cap.isOpened())
print("Frame count:", cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("FPS:", cap.get(cv2.CAP_PROP_FPS))

ret, frame = cap.read()
print("Read success:", ret)
if ret:
    print("Frame shape:", frame.shape)
else:
    print("⚠️ Could not read any frame.")

cap.release()

Opened: True
Frame count: 308.0
FPS: 30.0
Read success: True
Frame shape: (1440, 2732, 3)


In [ ]:
cap = cv2.VideoCapture("/content/video.mp4")

# Grab first frame just to know size
ok, sample = cap.read()
if not ok:
    raise RuntimeError("Video opened but returned no frame")

# Compute output parameters
sample = cv2.resize(sample, (640, 360))
h, w = sample.shape[:2]
fps = cap.get(cv2.CAP_PROP_FPS) or 25

# Initialise writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('task3.mp4', fourcc, fps, (w, h))

# 🔹 Re-open the video from the start instead of cap.set(...)
cap.release()
cap = cv2.VideoCapture("/content/video.mp4")

print("Writer ready:", w, h, "@", fps)


Writer ready: 640 360 @ 30.0


In [ ]:
transform=transforms.Compose([transforms.ToTensor()])
confidence_threshold=0.3
iou_threshold=0.4
target_classes=['person', 'dog', 'book', 'mouse', 'laptop', 'tv']

object_id=0
trackers=[]

def iou(box1,box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    box1Area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2Area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    return interArea / float(box1Area + box2Area - interArea + 1e-6)


In [ ]:
frame_id = 0
skip = 5
written_frames=0

while True :
    ret,frame =cap.read()
    if not ret:
        break
    frame_id += 1
    if frame is None or frame_id % skip != 0:  # Added check for empty frame
        continue
    frame = cv2.resize(frame, (640, 360))
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


    img=transform(frame_rgb)
    with torch.no_grad():
        preds=model([img])[0]

    keep = preds['scores'] > confidence_threshold
    boxes = preds['boxes'][keep].cpu().numpy()
    labels = preds['labels'][keep].cpu().numpy()
    scores = preds['scores'][keep].cpu().numpy()

    detections=[]
    for box,label,score in zip(boxes,labels,scores):
        if score>confidence_threshold:
            if label < len(COCO_INSTANCE_CATEGORY_NAMES):
                name = COCO_INSTANCE_CATEGORY_NAMES[label]

            if not target_classes or name in target_classes: # Fixed logic to check if name is in target_classes
                    detections.append((box,name,score)) # Append with name instead of label

    new_trackers=[]
    for box,name,score in detections:
        matched_id=None
        for t in trackers: # Iterate through trackers directly
            if t['label'] == name and iou(box, t['box']) > iou_threshold:
                matched_id = t['id']
                break
        if matched_id is None:
            object_id+=1
            matched_id=object_id
        new_trackers.append({'id':matched_id,'box':box,'label':name,'score':score})

    trackers =new_trackers

    for t in trackers :
        (x1, y1, x2, y2) = t['box'].astype(int)
        cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame_rgb,
                    f"ID {t['id']} - {t['label']} ({t['score']:.2f})",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

    frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
    out.write(frame_bgr)
    written_frames += 1

cap.release()
out.release()

cv2.destroyAllWindows()

print("✅ task3.mp4 saved successfully!") # Changed extension to .avi

✅ task3.mp4 saved successfully!


In [ ]:
out.write(frame)

In [ ]:
import os
print(os.path.exists("task3.mp4"))
print("Size:", os.path.getsize("task3.mp4")/1024, "KB")


True
Size: 1816.6298828125 KB


In [ ]:
!ls -lh /content | grep task3

-rw-r--r-- 1 root root 1.8M Oct  9 01:41 task3.mp4


In [ ]:
!ffmpeg -y -i /content/task3.mp4 -vcodec libx264 -pix_fmt yuv420p /content/final_task3.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
#!ffmpeg -i task3.avi -vcodec libx264 -crf 24 task3.mp4 -y


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from IPython.display import Video
Video("/content/final_task3.mp4", embed=True, width=640, height=360)